## Porting weights from PyTorch

The original ESM-1b model was written in PyTorch, which you can find [here](https://github.com/facebookresearch/esm). This notebook assumes you have PyTorch (`torch`) and the ESM library (`esm`) installed. Having run this, you'll download both the original model, and convert the weights into pure numPy arrays, stored in an HDF5 that are ready for use by Haiku. Note you don't *need* to run this notebook to run the conversion yourself, the provided `.h5` file should be enough!

In [1]:
# Allows notebook to be run as though its at root of the repo
# instead of inside /notebooks
import sys
sys.path.insert(0, '..')

from collections import defaultdict

# import pytorch and ESM
import torch, esm

In [2]:
# Load model. First time will take a while, it's downloading ~7GB
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

The following cell ports over all the weights needed for embedding generator and contact prediction into a two level `dict` of `ndarray`s, which can then be trivially converted by Haiku into model parameters. Note we're not copying over the weights for the RoBERTa head, as that provides the unsupervised learning signal only during training, and isn't used during inference.

It's a little un-elegant, but it gets the job done!

In [3]:
# if key doesn't exist, initialize value to {}
param_dict = defaultdict(lambda: {})

# copy weights for the initial embedding layers
param_dict['esm1b/embed']['embeddings'] = model.embed_tokens.weight.detach().numpy()
param_dict['esm1b/learned_positional_embeddings/embed']['embeddings'] = model.embed_positions.weight.detach().numpy()
param_dict['esm1b/emb_layer_norm_before']['scale'] = model.emb_layer_norm_before.weight.detach().numpy()
param_dict['esm1b/emb_layer_norm_before']['offset'] = model.emb_layer_norm_before.bias.detach().numpy()

# copy weights for each of the 33 transformer blocks
for layer_idx in range(33):
    # multiheaded self-attention weights
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/query']['w'] = model.layers[layer_idx].self_attn.q_proj.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/query']['b'] = model.layers[layer_idx].self_attn.q_proj.bias.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/key']['w'] = model.layers[layer_idx].self_attn.k_proj.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/key']['b'] = model.layers[layer_idx].self_attn.k_proj.bias.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/value']['w'] = model.layers[layer_idx].self_attn.v_proj.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/value']['b'] = model.layers[layer_idx].self_attn.v_proj.bias.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/linear']['w'] = model.layers[layer_idx].self_attn.out_proj.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_mha/linear']['b'] = model.layers[layer_idx].self_attn.out_proj.bias.detach().numpy()

    # weights for the linear layers 
    param_dict[f'esm1b/transformer_layer_{layer_idx}/fc1']['w'] = model.layers[layer_idx].fc1.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/fc1']['b'] = model.layers[layer_idx].fc1.bias.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/fc2']['w'] = model.layers[layer_idx].fc2.weight.detach().numpy().T
    param_dict[f'esm1b/transformer_layer_{layer_idx}/fc2']['b'] = model.layers[layer_idx].fc2.bias.detach().numpy()

    # weights for the layer norms 
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_attn_layer_norm']['scale'] = model.layers[layer_idx].self_attn_layer_norm.weight.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/self_attn_layer_norm']['offset'] = model.layers[layer_idx].self_attn_layer_norm.bias.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/final_layer_norm']['scale'] = model.layers[layer_idx].final_layer_norm.weight.detach().numpy()
    param_dict[f'esm1b/transformer_layer_{layer_idx}/final_layer_norm']['offset'] = model.layers[layer_idx].final_layer_norm.bias.detach().numpy()

# weights for the post-transformer blocks layer norm. Needed for final layer embeddings.
param_dict['esm1b/emb_layer_norm_after']['scale'] = model.emb_layer_norm_after.weight.detach().numpy()
param_dict['esm1b/emb_layer_norm_after']['offset'] = model.emb_layer_norm_after.bias.detach().numpy()

# weights for the contact prediction head
param_dict['contact_pred_head/regression']['w'] = model.contact_head.regression.weight.detach().numpy().T
param_dict['contact_pred_head/regression']['b'] = model.contact_head.regression.bias.detach().numpy()

Then, we just save this two-level dict of `numpy` arrays into a `.h5` file.

In [4]:
from esmjax import modelio

modelio.save_model(param_dict, "../weights/esm1b.h5")

The saved `.h5` should be ~2.5GB in size. This is smaller than the PyTorch download, as we're only saving the model parameters (not the state of the optimizer/other add-ons). If you divide 2.5GB by 4 bytes per param (as each param is saved in `float32`), we get ~650M, which is indeed the number of parameters